In [1]:
import pandas as pd
from datetime import datetime

In [2]:
msp_data = pd.read_csv("mu_clean.csv",index_col=0)

msp_data['Fault Date'] = pd.to_datetime(msp_data['ZULU_Time'].str.split(" ", n = 2, expand = True)[1])

msp_data['Fault Time'] = pd.to_datetime(msp_data['ZULU_Time'].str.split(" ", n = 2, expand = True)[2], format = '%H:%M:%S:%f' ,errors = 'ignore')


C:\Users\rando\.conda\envs\New_Dev\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
msp_data.head()

,Aircraft,Squadron,Lot,MSP,ZULU_Time,Flight_Mode,Fault Date,Fault Time
0,1,HH,100,ZJMZTL,01-APR-2014 00:25:39:00,EngineTurn,2014-04-01,00:25:39:00
1,1,HH,100,JDJDMQ,01-APR-2014 00:25:39:00,EngineTurn,2014-04-01,00:25:39:00
2,1,HH,100,RTVBHP,01-APR-2014 00:25:39:00,EngineTurn,2014-04-01,00:25:39:00
3,1,HH,100,RTUXHP,01-APR-2014 00:25:39:00,EngineTurn,2014-04-01,00:25:39:00
4,1,HH,100,RTUXMZ,01-APR-2014 00:25:39:00,EngineTurn,2014-04-01,00:25:39:00


In [4]:
maf_data = pd.read_csv("maf_clean.csv", index_col = 0)

maf_data['Received Date'] = pd.to_datetime(maf_data['Received Date'])

maf_data['Completion Date'] = pd.to_datetime(maf_data['Completion Date'])

In [5]:
maf_data.head()

,Job Code,Aircraft,Transaction Code,Malfunction Code,Action Taken Code,Description of Problem,Correction of Problem,Received Date,Completion Date,Corrosion,Bare Metal,Corrosion Prevention Treatment,Routine Maintenance,Unscheduled Maintenance,Mission-Related Maintenance,Failure
0,0NGHY44WC8118573,42,11,0,N,Perform system or component checks,Completed the component or system test,2012-04-26,2014-02-27,False,False,False,True,False,False,False
1,0NGHY45WC8118591,42,11,0,N,Perform system or component checks,Completed the component or system test,2012-04-26,2014-02-27,False,False,False,True,False,False,False
2,0NGHY46WC8118581,42,11,0,N,Perform a periodic inspection,Completed the inspection,2012-04-26,2014-02-27,False,False,False,True,False,False,False
3,0NGHY47WC8118577,42,11,0,N,Perform a periodic inspection,Completed the inspection,2012-04-26,2014-02-27,False,False,False,True,False,False,False
4,0NGHY48WC8118586,42,11,0,N,Perform a periodic inspection,Completed the inspection,2012-04-26,2014-02-27,False,False,False,True,False,False,False


In [6]:
corrosion_actions = maf_data[maf_data['Corrosion']| maf_data['Bare Metal'] | maf_data['Corrosion Prevention Treatment']]

In [7]:
corrosion_actions["action_month"] = pd.DatetimeIndex(corrosion_actions['Completion Date']).month

corrosion_actions["action_year"] = pd.DatetimeIndex(corrosion_actions['Completion Date']).year

C:\Users\rando\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\rando\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
msp_data['action_year'] = pd.DatetimeIndex(msp_data['Fault Date']).year

In [9]:
msp_data.head()

,Aircraft,Squadron,Lot,MSP,ZULU_Time,Flight_Mode,Fault Date,Fault Time,action_year
0,1,HH,100,ZJMZTL,01-APR-2014 00:25:39:00,EngineTurn,2014-04-01,00:25:39:00,2014
1,1,HH,100,JDJDMQ,01-APR-2014 00:25:39:00,EngineTurn,2014-04-01,00:25:39:00,2014
2,1,HH,100,RTVBHP,01-APR-2014 00:25:39:00,EngineTurn,2014-04-01,00:25:39:00,2014
3,1,HH,100,RTUXHP,01-APR-2014 00:25:39:00,EngineTurn,2014-04-01,00:25:39:00,2014
4,1,HH,100,RTUXMZ,01-APR-2014 00:25:39:00,EngineTurn,2014-04-01,00:25:39:00,2014


In [10]:
combined_data = corrosion_actions.merge(msp_data[['Aircraft', 'action_year', 'MSP', 'Flight_Mode', 'Fault Date']], how = 'left', on = ['Aircraft', 'action_year'])

In [11]:
combined_data['before_window'] = combined_data['Completion Date'] - pd.DateOffset(months = 1)

combined_data['after_window'] = combined_data['Completion Date'] + pd.DateOffset(months = 1)

In [12]:
# combined_data = combined_data[(combined_data['Fault Date'] > combined_data['before_window'])] 
# combined_data = combined_data[(combined_data['Fault Date'] < combined_data['after_window'])]

In [13]:
combined_data['Pre'] = combined_data['Fault Date'] < combined_data['Completion Date']

In [14]:
combined_data.head()

,Job Code,Aircraft,Transaction Code,Malfunction Code,Action Taken Code,Description of Problem,Correction of Problem,Received Date,Completion Date,Corrosion,...,Mission-Related Maintenance,Failure,action_month,action_year,MSP,Flight_Mode,Fault Date,before_window,after_window,Pre
0,0NGHY4HWC8118595,42,11,0,N,Perform system or component checks,Completed the component or system test,2012-04-26,2014-02-27,True,...,False,False,2,2014,VBHP,GrndPwr,2014-04-01,2014-01-27,2014-03-27,False
1,0NGHY4HWC8118595,42,11,0,N,Perform system or component checks,Completed the component or system test,2012-04-26,2014-02-27,True,...,False,False,2,2014,MZUXNV,GrndPwr,2014-04-01,2014-01-27,2014-03-27,False
2,0NGHY4HWC8118595,42,11,0,N,Perform system or component checks,Completed the component or system test,2012-04-26,2014-02-27,True,...,False,False,2,2014,MZUXBM,GrndPwr,2014-04-01,2014-01-27,2014-03-27,False
3,0NGHY4HWC8118595,42,11,0,N,Perform system or component checks,Completed the component or system test,2012-04-26,2014-02-27,True,...,False,False,2,2014,MZQGTL,GrndPwr,2014-04-01,2014-01-27,2014-03-27,False
4,0NGHY4HWC8118595,42,11,0,N,Perform system or component checks,Completed the component or system test,2012-04-26,2014-02-27,True,...,False,False,2,2014,MZVBRT,GrndPwr,2014-04-01,2014-01-27,2014-03-27,False


Removing MSP codes within service window

In [15]:
#combined_data = combined_data[(combined_data['Fault Date'] > combined_data['Completion Date']) | (combined_data['Fault Date'] < combined_data['Received Date'])]

In [16]:
combined_data.head()

,Job Code,Aircraft,Transaction Code,Malfunction Code,Action Taken Code,Description of Problem,Correction of Problem,Received Date,Completion Date,Corrosion,...,Mission-Related Maintenance,Failure,action_month,action_year,MSP,Flight_Mode,Fault Date,before_window,after_window,Pre
0,0NGHY4HWC8118595,42,11,0,N,Perform system or component checks,Completed the component or system test,2012-04-26,2014-02-27,True,...,False,False,2,2014,VBHP,GrndPwr,2014-04-01,2014-01-27,2014-03-27,False
1,0NGHY4HWC8118595,42,11,0,N,Perform system or component checks,Completed the component or system test,2012-04-26,2014-02-27,True,...,False,False,2,2014,MZUXNV,GrndPwr,2014-04-01,2014-01-27,2014-03-27,False
2,0NGHY4HWC8118595,42,11,0,N,Perform system or component checks,Completed the component or system test,2012-04-26,2014-02-27,True,...,False,False,2,2014,MZUXBM,GrndPwr,2014-04-01,2014-01-27,2014-03-27,False
3,0NGHY4HWC8118595,42,11,0,N,Perform system or component checks,Completed the component or system test,2012-04-26,2014-02-27,True,...,False,False,2,2014,MZQGTL,GrndPwr,2014-04-01,2014-01-27,2014-03-27,False
4,0NGHY4HWC8118595,42,11,0,N,Perform system or component checks,Completed the component or system test,2012-04-26,2014-02-27,True,...,False,False,2,2014,MZVBRT,GrndPwr,2014-04-01,2014-01-27,2014-03-27,False


Remove MSP codes that do not occur frequently

In [17]:
combined_data['row'] = True

In [18]:
combined_data.head()

,Job Code,Aircraft,Transaction Code,Malfunction Code,Action Taken Code,Description of Problem,Correction of Problem,Received Date,Completion Date,Corrosion,...,Failure,action_month,action_year,MSP,Flight_Mode,Fault Date,before_window,after_window,Pre,row
0,0NGHY4HWC8118595,42,11,0,N,Perform system or component checks,Completed the component or system test,2012-04-26,2014-02-27,True,...,False,2,2014,VBHP,GrndPwr,2014-04-01,2014-01-27,2014-03-27,False,True
1,0NGHY4HWC8118595,42,11,0,N,Perform system or component checks,Completed the component or system test,2012-04-26,2014-02-27,True,...,False,2,2014,MZUXNV,GrndPwr,2014-04-01,2014-01-27,2014-03-27,False,True
2,0NGHY4HWC8118595,42,11,0,N,Perform system or component checks,Completed the component or system test,2012-04-26,2014-02-27,True,...,False,2,2014,MZUXBM,GrndPwr,2014-04-01,2014-01-27,2014-03-27,False,True
3,0NGHY4HWC8118595,42,11,0,N,Perform system or component checks,Completed the component or system test,2012-04-26,2014-02-27,True,...,False,2,2014,MZQGTL,GrndPwr,2014-04-01,2014-01-27,2014-03-27,False,True
4,0NGHY4HWC8118595,42,11,0,N,Perform system or component checks,Completed the component or system test,2012-04-26,2014-02-27,True,...,False,2,2014,MZVBRT,GrndPwr,2014-04-01,2014-01-27,2014-03-27,False,True


In [24]:
msp_counts = combined_data[['Job Code']].groupby(['Job Code', 'MSP', 'Pre']).count().reset_index()
# msp_counts = combined_data[['Pre']].groupby(['Job Code', 'MSP', 'Pre']).count().reset_index()
# msp_counts = combined_data[['row']].groupby(['Job Code', 'MSP', 'Pre']).count().reset_index()
# msp_counts = combined_data[['MSP']].groupby(['Job Code', 'MSP', 'Pre']).count().reset_index()


MemoryError: Unable to allocate 11.2 GiB for an array with shape (5, 300943423) and data type datetime64[ns]

Finding codes to keep based on frequency in this dataset

In [ ]:
msp_pre = combined_data.query('Pre == True').groupby('MSP')[['row']].count().reset_index().rename(columns = {'row': 'pre'})

In [ ]:
msp_post = combined_data.query('Pre == False').groupby('MSP')[['row']].count().reset_index().rename(columns = {'row': 'post'})

join on msp, calculate pre/post diff

In [ ]:
msp_diff = msp_post.merge(msp_pre, how = 'left', on = 'MSP')

In [ ]:
msp_diff['diff'] = msp_diff['pre'] - msp_diff['post']

In [ ]:
msp_diff.sort_values('diff', ascending=False).reset_index().head(20).plot.bar(x = 'MSP', y = 'diff');

In [ ]:
msp_interest = msp_diff.sort_values('diff', ascending=False).reset_index().head(20)['MSP']

In [ ]:
msp_interest

In [ ]:
msp_counts[msp_counts['MSP'].isin(msp_interest)].to_csv('MSP_freq.csv', index = False)

In [ ]:
corrosion = msp_counts[msp_counts['MSP'].isin(msp_interest)].query('Pre').drop('Pre', axis = 1).pivot(index = 'Job Code', columns = 'MSP', values = 'row').fillna(0.0).reset_index()

corrosion['corrosion'] = True

corrosion.drop('Job Code', axis = 1, inplace = True)

In [ ]:
corrosion.head()

Building a dataset of non-corrosive corrective actions

In [ ]:
non_corrosion_actions = maf_data[~(maf_data['Corrosion'] | maf_data['Bare Metal'] | maf_data['Corrosion Prevention Treatment'])].sample(60000, axis = 0, random_state = 35)

non_corrosion_actions["action_month"] = pd.DatetimeIndex(non_corrosion_actions['Completion Date']).month

non_corrosion_actions["action_year"] = pd.DatetimeIndex(non_corrosion_actions['Completion Date']).year

combined_data_non = non_corrosion_actions.merge(msp_data[['Aircraft', 'action_year', 'MSP', 'Flight_Mode', 'Fault Date']], how = 'left', on = ['Aircraft', 'action_year'])

combined_data_non['before_window'] = combined_data_non['Completion Date'] - pd.DateOffset(months = 1)

combined_data_non['after_window'] = combined_data_non['Completion Date'] + pd.DateOffset(months = 1)

combined_data_non = combined_data_non[(combined_data_non['Fault Date'] > combined_data_non['before_window']) & (combined_data_non['Fault Date'] < combined_data_non['after_window'])]

combined_data_non['Pre'] = combined_data_non['Fault Date'] < combined_data_non['Completion Date']

combined_data_non = combined_data_non[(combined_data_non['Fault Date'] > combined_data_non['Completion Date']) | (combined_data_non['Fault Date'] < combined_data_non['Received Date'])]

In [ ]:
combined_data_non['row'] = True

non_msp_counts = combined_data_non[['Job Code', 'MSP', 'Pre', 'row']].groupby(['Job Code', 'MSP', 'Pre']).count().reset_index()

In [ ]:
non_corrosion = non_msp_counts[non_msp_counts['MSP'].isin(msp_interest)].query('Pre').drop('Pre', axis = 1).pivot(index = 'Job Code', columns = 'MSP', values = 'row').fillna(0.0).reset_index()

non_corrosion['corrosion'] = False

non_corrosion.drop('Job Code', axis = 1, inplace = True)

In [ ]:
non_corrosion.shape

In [ ]:
corrosion.shape

In [ ]:
non_corrosion.append(corrosion).to_csv('../clean/predict_corrosive.csv', index = True)